# Vaccine Trail Analytics Dataset Creation

#### Data Ingestion

In [1]:
import pandas as pd
import numpy as np
brief_summaries = pd.read_csv('brief_summaries.csv', on_bad_lines='skip')
studies = pd.read_csv('studies.csv', on_bad_lines='skip')
designs = pd.read_csv('designs.csv', on_bad_lines='skip')
interventions=pd.read_csv('interventions.csv',usecols=['nct_id','intervention_type','name','description'])
browse_conditions = pd.read_csv('browse_conditions.csv')
browse_interventions = pd.read_csv("browse_interventions.csv")
keywords = pd.read_csv('keywords.csv')
conditions = pd.read_csv('conditions.csv')
design_groups = pd.read_csv('design_groups.csv')
countries = pd.read_csv('countries.csv')


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (47,48,53,68) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


#### Initial Filtering

In [29]:
# All Vaccine related information

studies['brief_title'] = studies['brief_title'].astype(str)
studies['official_title'] = studies['official_title'].astype(str)

# Defining the regex pattern for vaccine-related terms
pattern = r'\bvaccin(?:e|ation|es|ations)?\b'


# Filtering the dataset for trials containing relevant terms in the brief_title or official_title
vaccine_related_df = studies[
    studies['brief_title'].str.contains(pattern, case=False, na=False, regex=True) |
    studies['official_title'].str.contains(pattern, case=False, na=False, regex=True)
]

# Ensuring study_first_posted_date is in datetime format
vaccine_related_df['study_first_posted_date'] = pd.to_datetime(vaccine_related_df['study_first_posted_date'], errors='coerce')


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [30]:

# Load the tables


# Filter for vaccine-related information
brief = brief_summaries[brief_summaries['description'].str.lower().str.contains('vaccine')]

# Join the tables
vaccine_join = pd.merge(brief, studies, on='nct_id')
vaccine_join = pd.merge(vaccine_join, designs[['nct_id', 'primary_purpose']], on='nct_id')

# Select the relevant columns
vaccine_join = vaccine_join[['nct_id', 'start_date', 'completion_date', 'study_type', 'primary_purpose', 'brief_title', 'official_title', 'description']]

# Save the result to a new CSV file (optional)
# vaccine_join.to_csv('vaccine_join.csv', index=False)

# Display the result
H_df = vaccine_join.copy()
H_df = H_df[H_df['nct_id'].str.startswith('NCT')]



In [39]:
def combined_data(data1,data2,col,condition):
    data= pd.merge(data1, data2,on=col,how=condition)
    data = data.drop_duplicates(subset=[col])
    return data
# Filetered tables individually with conditions
# Designs Table
condition_design = (designs['primary_purpose'] == 'PREVENTION') | (designs['primary_purpose'] == 'TREATMENT')| (pd.isna(designs['primary_purpose']))
designs_1=designs[condition_design]
designs_1=designs_1.drop_duplicates(subset='nct_id')

#Intervention Table
condition_int = interventions['intervention_type']=='BIOLOGICAL'
interventions_1=interventions[condition_int]
interventions_1=interventions_1.drop_duplicates(subset='nct_id')

#Studies Table
condition_st = (studies['study_type']=='INTERVENTIONAL') | (studies['study_type']== 'OBSERVATIONAL')|(studies['study_type']== 'Observational [Patient Registry]')
studies_1 = studies[condition_st]

#combined tables
nct_id= combined_data(interventions_1,studies_1,'nct_id', 'inner')
common_nct_id= combined_data(designs_1,nct_id,'nct_id','inner')
S_df = common_nct_id.copy()

### Merging Different Parts 

In [40]:
matched_df = pd.merge(vaccine_related_df, H_df, on='nct_id', how='outer')
columns_to_converge = ['start_date', 'completion_date', 'study_type', 'brief_title', 'official_title']

# Iterate through the columns and combine 'x' and 'y' versions
for col in columns_to_converge:
    matched_df[col] = matched_df[f'{col}_y'].combine_first(matched_df[f'{col}_x'])
    matched_df.drop([f'{col}_x', f'{col}_y'], axis=1, inplace=True)

print(matched_df.shape)

(10753, 73)


In [49]:
final_df = pd.merge(matched_df, S_df, on='nct_id', how='outer', suffixes=('', '_y'))

# Identify all overlapping columns (excluding 'nct_id')
overlapping_columns = [col for col in final_df.columns if col.endswith('_y') and col[:-2] in final_df.columns]

# Function to handle column convergence
def converge_columns(df, col):
    x_col = col
    y_col = f'{col}_y'
    
    # Case 1: Values in x are equal to y, set y to None
    df.loc[df[x_col] == df[y_col], y_col] = None
    
    # Case 2: Values only available in x column, keep them the same (No action needed)
    
    # Case 3: Values only available in y column, copy those to x
    df.loc[df[x_col].isna(), x_col] = df[y_col]
    
    # Set y to None where x is not null (covers both case 1 and 2)
    df.loc[df[x_col].notna(), y_col] = None
    
    # Drop the y column
    df.drop(columns=[y_col], inplace=True)
    
    return df

# Apply the convergence function to all overlapping columns
for col in overlapping_columns:
    final_df = converge_columns(final_df, col[:-2])


### Final_df is preliminary merged dataset out of all dataset which contains false positives

In [ ]:
final_df.rename(columns={'study_type_x': 'study_type', 'primary_purpose_x': 'primary_purpose'}, inplace=True)


### Hence, combining extra rows from some parts on which we will filter more

In [55]:
vaccine_nct_ids = set(vaccine_related_df['nct_id'])

# Filter out rows from S_df and H_df where nct_id is in vaccine_nct_ids
S_df_filtered = S_df[~S_df['nct_id'].isin(vaccine_nct_ids)]
H_df_filtered = H_df[~H_df['nct_id'].isin(vaccine_nct_ids)]

# Combine the two filtered datasets
combined_df = pd.concat([S_df_filtered, H_df_filtered], ignore_index=True)

### Additional Filtering

In [56]:
# Applying filtering techniques 

vaccine_list = [
    'hepatitis', 'diphtheria', 'flu', 'hib', 'hpv', 'measles', 'meningococcal', 'mumps', 
    'polio', 'rsv', 'pneumococcal', 'tetanus', 'rubella', 'hep b', 'mmr', 'vaccinia', 'dtap', 
    'hiv', 'bcg', 'hiv-1', 'tuberculosis', 'norovirus', 'virus', 'corona', 'dpt', 'ipv', 
    'covid-19', 'pneumonia', 'infection', 'rabies', 'vaccine', 'vaccination'
]

# Remove list based on frequent words irrelevant to vaccine studies
remove_list = [
    'inhibitor' , 'mesenchymal' , 'autologous' , 'osteoarthritis', 'macular', 'immunoglobulin', 'transplantation' , 'lymphocytes' 
]
    


# Function to filter rows based on vaccine_list and remove_list
def filter_vaccine_studies(df, vaccine_list, remove_list):
    def contains_keywords(text, keywords):
        if pd.isna(text):
            return False
        text = text.lower()
        return any(keyword in text for keyword in keywords)
    
    # Filter rows that contain vaccine-related keywords and do not contain remove_list keywords
    filtered_df = df[
        df.apply(lambda row: (
            contains_keywords(row['name'], vaccine_list) or 
            contains_keywords(row['description'], vaccine_list) or 
            contains_keywords(row['brief_title'], vaccine_list) or 
            contains_keywords(row['official_title'], vaccine_list)
        ) and not (
            contains_keywords(row['name'], remove_list) or 
            contains_keywords(row['description'], remove_list) or 
            contains_keywords(row['brief_title'], remove_list) or 
            contains_keywords(row['official_title'], remove_list)
        ), axis=1)
    ]
    
    return filtered_df

# Apply the filter function
filtered_vaccine_studies = filter_vaccine_studies(combined_df, vaccine_list, remove_list)


### Creating a true positive dataset that contains vaccine related informational rows from all datasets

In [57]:
#Getting NCTids of true positives - calculated by entries present in atelast 2 of the 3 datasets

vaccine_nct_ids = set(vaccine_related_df['nct_id'])
H_nct_ids = set(H_df['nct_id'])
S_nct_ids = set(S_df['nct_id'])

# Function to check if nct_id is present in at least 2 datasets
def is_in_at_least_two(nct_id, sets):
    count = sum(1 for s in sets if nct_id in s)
    return count >= 2

# Filter the final dataframe
true_positive = final_df[final_df['nct_id'].apply(lambda nct_id: is_in_at_least_two(nct_id, [vaccine_nct_ids, H_nct_ids, S_nct_ids]))]


### Filtering that from prelimary merged dataset 

In [58]:
final_df_filtered_nct_ids = set(true_positive['nct_id'])
filtered_vaccine_studies_nct_ids = set(filtered_vaccine_studies['nct_id'])

# Combine the nct_ids from both datasets
combined_nct_ids = final_df_filtered_nct_ids.union(filtered_vaccine_studies_nct_ids)

# Filter out rows from final_df that are present in either final_df_filtered or filtered_vaccine_studies
final_df_filtered_out = final_df[final_df['nct_id'].isin(combined_nct_ids)]


### Filtering needed columns and adding new columns from other datasets / aggregating data wherever needed

In [62]:
df = final_df_filtered_out[["nct_id","start_date","start_date_type","completion_date_type", "completion_date" ,'primary_completion_date', "primary_completion_date_type","target_duration", "phase","overall_status", "study_type","source" ,"source_class","brief_title", 
"official_title","enrollment","acronym" ,"why_stopped", "number_of_groups", "number_of_arms" ,"enrollment_type",'intervention_type','name', 'description','intervention_model_description','primary_purpose']]

In [63]:

# Aggregating the browse_conditions data
aggregated_conditions = browse_conditions.groupby('nct_id').agg({
    'downcase_mesh_term': lambda x: ' '.join(x.unique()),
    'mesh_type': lambda x: ' '.join(x.unique())
}).reset_index()

# Renaming the columns as per the provided mapping
aggregated_conditions.columns = ['nct_id', 'mesh_conditions', 'mesh_term_type_browse_conditions']

# Merging with final_filtered on nct_id using left join to keep all rows in final_filtered
df1 = df.merge(aggregated_conditions, on='nct_id', how='left')

In [66]:
# Aggregating the browse_conditions data
aggregated_interventions = browse_interventions.groupby('nct_id').agg({
    'downcase_mesh_term': lambda x: ' '.join(x.unique()),
    'mesh_type': lambda x: ' '.join(x.unique())
}).reset_index()

# Renaming the columns as per the provided mapping
aggregated_interventions.columns = ['nct_id', 'mesh_interventions', 'mesh_term_type_browse_interventions']

# Merging with final_filtered on nct_id
df2 = df1.merge(aggregated_interventions, on='nct_id', how='left')


In [67]:
# Aggregating the keywords data
aggregated_keywords = keywords.groupby('nct_id').agg({
    'downcase_name': lambda x: ' '.join(x.unique())
}).reset_index()

# Renaming the columns as per the provided mapping
aggregated_keywords.columns = ['nct_id', 'keywords']


# Merging with final_filtered on nct_id using left join to keep all rows in final_filtered
df3 = df2.merge(aggregated_keywords, on='nct_id', how='left')


In [68]:
# Aggregating the conditions data
aggregated_conditions = conditions.groupby('nct_id').agg({
    'downcase_name': lambda x: ' '.join(x.unique())
}).reset_index()

# Renaming the columns as per the provided mapping
aggregated_conditions.columns = ['nct_id', 'conditions']


# Merging with final_filtered on nct_id using left join to keep all rows in final_filtered
df4 = df3.merge(aggregated_conditions, on='nct_id', how='left')


In [69]:
# Selecting and renaming columns as per the provided mapping
designs_selected = designs[['nct_id', 'intervention_model', 'observational_model', 'allocation', 
                            'time_perspective', 'masking', 'masking_description', 
                            'subject_masked', 'caregiver_masked', 'investigator_masked', 
                            'outcomes_assessor_masked', 'intervention_model_description']]
designs_selected.columns = [
    'nct_id', 'interventional_mode', 'observational_mode', 'design_allocation', 
    'design_time_perspective', 'design_masking', 'design_masking_description', 
    'design_subject_masked', 'design_caregiver_masked', 'design_investigator_masked', 
    'design_outcome_assessor_masked', 'interventional_mode_description'
]

# Merging with final_filtered on nct_id using left join to keep all rows in final_filtered
df5 = df4.merge(designs_selected, on='nct_id', how='left')


In [70]:
design_groups['group_type'] = design_groups['group_type'].fillna('')

# Aggregating the design_groups data
aggregated_design_groups = design_groups.groupby('nct_id').agg({
    'group_type': lambda x: ' '.join(x.unique())
}).reset_index()

# Renaming the columns as per the provided mapping
aggregated_design_groups.columns = ['nct_id', 'design_groups_type']

# Merging with final_filtered on nct_id using left join to keep all rows in final_filtered
df5 = df5.merge(aggregated_design_groups, on='nct_id', how='left')



In [71]:

# Handling NaN values by replacing them with an empty string
countries['name'] = countries['name'].fillna('')

# Aggregating the countries data
aggregated_countries = countries.groupby('nct_id').agg({
    'name': lambda x: ' '.join(x.unique())
}).reset_index()

# Renaming the columns as per the provided mapping
aggregated_countries.columns = ['nct_id', 'countries']

# Merging with final_filtered on nct_id using left join to keep all rows in final_filtered
df6 = df5.merge(aggregated_countries, on='nct_id', how='left')


In [72]:
#df6 is master dataframe of all vaccine related clinical trials data

df6.columns

Index(['nct_id', 'start_date', 'start_date_type', 'completion_date_type',
       'completion_date', 'primary_completion_date',
       'primary_completion_date_type', 'target_duration', 'phase',
       'overall_status', 'study_type', 'source', 'source_class', 'brief_title',
       'official_title', 'enrollment', 'acronym', 'why_stopped',
       'number_of_groups', 'number_of_arms', 'enrollment_type',
       'intervention_type', 'name', 'description',
       'intervention_model_description', 'primary_purpose', 'mesh_conditions',
       'mesh_term_type_browse_conditions', 'mesh_interventions',
       'mesh_term_type_browse_interventions', 'keywords', 'conditions',
       'interventional_mode', 'observational_mode', 'design_allocation',
       'design_time_perspective', 'design_masking',
       'design_masking_description', 'design_subject_masked',
       'design_caregiver_masked', 'design_investigator_masked',
       'design_outcome_assessor_masked', 'interventional_mode_description',
   